<a href="https://colab.research.google.com/github/A-Drey2002/HyperChat/blob/main/Installing_langchain%2C_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Set up Virtual environment

In [ ]:
import os
# Access environment variables
tracing_enabled = os.getenv('LANGCHAIN_TRACING_V2')
api_key = os.getenv('LANGCHAIN_API_KEY')

# Use the environment variables
print(f"Tracing Enabled: {tracing_enabled}")
print(f"API Key: {api_key}")



Tracing Enabled: None
API Key: None


### Creating LLM chain via Ollama integration (local and open source llama2)

In [ ]:
# Set up a local ollama instance
%pip install langchain
%pip install langchain_community
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Verify successful importation
try:
    from langchain_community.llms import Ollama
    llm = Ollama(model="llama2")
    print("Module imported successfully.")
except ModuleNotFoundError as e:
    print(f"Error: {e}")


Module imported successfully.


In [ ]:
# Test run
#llm.invoke("how can langsmith help with testing?")

In [ ]:
# Prompt Engineering
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [ ]:
# Convert output to string
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()


In [ ]:
# Create LLM chain
chain = prompt | llm | output_parser

In [ ]:
# invoke chain after prompting
# chain.invoke({"input": "How can langsmith help with testing?"})

## Retrieval System


### Authenticate and fetch documents from Google Drive

In [ ]:
# install pydrive library
%pip install pydrive\

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'pydrive\\': Expected end or semicolon (after name and no valid version specifier)
    pydrive\
           ^
Hint: It looks like a path. File 'pydrive\' does not exist.

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Import libraries
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials



In [ ]:
# Specify the path to the JSON key files
client_secrets_path = r"C:\Users\DELL\Desktop\Capstone\client_secret_715081280632-bo6ptf64unmpe7r9jkc79v0rbcm7323f.apps.googleusercontent.com.json"
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'C:\Users\DELL\Desktop\Capstone\HyperChat Service Account Key.json')

In [ ]:
# Authenticate and create the PyDrive client
gauth = GoogleAuth()
gauth.LoadClientConfigFile(client_secrets_path)
gauth.LocalWebserverAuth() # creates local webserver and automatically handles authentication
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=715081280632-bo6ptf64unmpe7r9jkc79v0rbcm7323f.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [ ]:
# Fetch a file by it's ID
file_id = "1uuNNsi3mXXN1VvHBP3dvSmoyATOxiCvo"
file = drive.CreateFile({'id': file_id})

# Download the file to local Machine
file.GetContentFile('document.txt')

In [ ]:
# Try reading the file with different encodings
encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
document_content = None

for encoding in encodings:
    try:
        with open('document.txt', 'r', encoding=encoding) as f:
            document_content = f.read()
        break  # If reading is successful, break out of the loop
    except UnicodeDecodeError:
        continue  # Try the next encoding

if document_content is None:
    raise UnicodeDecodeError("Unable to read the file with tried encodings.")


In [ ]:
# Create a document object
from langchain.docstore.document import Document
doc = Document(page_content=document_content, metadata={})

In [ ]:
# Split the document
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents([doc])

In [ ]:
# Extract texts for embedding
texts = [d.page_content for d in documents]

In [ ]:
# Import Embedding Model
from langchain_community.embeddings import OllamaEmbeddings
embedding_model = OllamaEmbeddings() # instantiate the embeddings object

In [ ]:
# Embed the documents
embedded_texts = embedding_model.embed_documents(texts)

In [ ]:
# Create metadata for the embeddings
metadata = [{"id": i, "text": text} for i, text in enumerate(texts)]

In [ ]:
# Pair each text with its corresponding embedding
# text_embedding_pairs = list(zip(texts, embedded_texts))

### Index File into a vectorstore

In [ ]:
# Import and install FAISS Vectorstore
from langchain_community.vectorstores import FAISS
%pip install faiss-cpu


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Build the vector index
vector = FAISS.from_embeddings(texts, embedded_texts, metadatas=metadata)
# Save the FAISS index
vector.save_local(r"C:\Users\DELL\Desktop\Capstone")

ValueError: too many values to unpack (expected 2)

## Creating the Retrieval Chain

In [ ]:
# Prompt | Retrieval Doc | LLM | Answer

from langchain.chains.combine_documents import create_stuff_documents_chain
prompt = ChatPromptTemplate.from_template("Answer the following question based only on the provided context:")
